In [1]:
import kernel_sidecar
kernel_sidecar.__version__

'0.5.5'

In [2]:
import json
connection_info = json.loads(open('conn.json').read())
connection_info

{'transport': 'tcp',
 'signature_scheme': 'hmac-sha256',
 'key': 'd3bafc903c6e4efc961f0223cd1d9cc1',
 'session': None,
 'shell_port': 50001,
 'stdin_port': 50002,
 'iopub_port': 50003,
 'hb_port': 50004,
 'control_port': 50005,
 'ip': '0.0.0.0',
 'kernel_name': 'rust-1.70'}

In [3]:
# Example of sending kernel info request and seeing which messages come back over the wire
# in this case status (kernel busy), kernel_info_reply, and status (kernel idle)
from kernel_sidecar.client import KernelSidecarClient
from kernel_sidecar.handlers.debug import DebugHandler

async with KernelSidecarClient(connection_info) as client:
    handler = DebugHandler()
    action = client.kernel_info_request(handlers=[handler])
    await action

In [4]:
handler.counts

defaultdict(int, {'status': 2, 'kernel_info_reply': 1})

In [5]:
# Turn on debug logs with structlog formatting for very verbose logging
# Note: just setup_logging() is enough in regular python apps, but in Jupyter we need to
# explicitly add a StreamHandler :man-shrugging:
import logging
logger = logging.getLogger('kernel_sidecar')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

from kernel_sidecar.log_utils import setup_logging
setup_logging()

In [6]:
async with KernelSidecarClient(connection_info) as client:
    await client.kernel_info_request()

Sent kernel_info_request to kernel


2023-09-05T19:53:34.087302Z [debug    ] Sent kernel_info_request to kernel [kernel_sidecar.client] filename=client.py func_name=send lineno=184 message=Sent kernel_info_request to kernel


Channel watcher started


2023-09-05T19:53:34.089671Z [debug    ] Channel watcher started        [kernel_sidecar.client] channel=iopub filename=client.py func_name=watch_channel lineno=312 message=Channel watcher started


Channel watcher started


...

2023-09-05T19:53:34.104915Z [debug    ] Message status on iopub        [kernel_sidecar.client] channel=iopub filename=client.py func_name=_watch_channel_for_messages lineno=398 message=Message status on iopub


Message kernel_info_reply on shell


2023-09-05T19:53:34.108276Z [debug    ] Message kernel_info_reply on shell [kernel_sidecar.client] channel=shell filename=client.py func_name=_watch_channel_for_messages lineno=398 message=Message kernel_info_reply on shell


Message status on iopub


2023-09-05T19:53:34.112039Z [debug    ] Message status on iopub        [kernel_sidecar.client] channel=iopub filename=client.py func_name=_watch_channel_for_messages lineno=398 message=Message status on iopub


In [7]:
from kernel_sidecar.nb_builder import NotebookBuilder, SimpleOutputHandler
from kernel_sidecar.models.notebook import Notebook, CodeCell

cell = CodeCell(id='1', source='println!("Hello Rust")')
nb = Notebook(cells=[cell])
builder = NotebookBuilder(nb)


async with KernelSidecarClient(connection_info) as client:
    handler = SimpleOutputHandler(client, cell.id, builder)
    await client.execute_request(code=cell.source, handlers=[handler])

handler

Sent execute_request to kernel


2023-09-05T19:53:34.152771Z [debug    ] Sent execute_request to kernel [kernel_sidecar.client] filename=client.py func_name=send lineno=184 message=Sent execute_request to kernel


Channel watcher started


2023-09-05T19:53:34.154861Z [debug    ] Channel watcher started        [kernel_sidecar.client] channel=iopub filename=client.py func_name=watch_channel lineno=312 message=Channel watcher started


Channel watcher started


...

2023-09-05T19:53:34.510440Z [debug    ] Message execute_result on iopub [kernel_sidecar.client] channel=iopub filename=client.py func_name=_watch_channel_for_messages lineno=398 message=Message execute_result on iopub


Message execute_reply on shell


2023-09-05T19:53:34.514886Z [debug    ] Message execute_reply on shell [kernel_sidecar.client] channel=shell filename=client.py func_name=_watch_channel_for_messages lineno=398 message=Message execute_reply on shell


Message status on iopub


2023-09-05T19:53:34.517887Z [debug    ] Message status on iopub        [kernel_sidecar.client] channel=iopub filename=client.py func_name=_watch_channel_for_messages lineno=398 message=Message status on iopub


In [8]:
builder.nb

Notebook(nbformat=4, nbformat_minor=5, metadata={}, cells=[CodeCell(id='1', source='println!("Hello Rust")', metadata={}, cell_type='code', execution_count=7, outputs=[StreamContent(output_type='stream', name='stdout', text='Hello Rust\n'), ExecuteResultContent(output_type='execute_result', execution_count=7, data={'text/plain': '()'}, metadata={})])])

In [9]:
import pprint
pprint.pprint(builder.nb.dict())

{'cells': [{'cell_type': 'code',
            'execution_count': 7,
            'id': '1',
            'metadata': {},
            'outputs': [{'name': 'stdout',
                         'output_type': 'stream',
                         'text': 'Hello Rust\n'},
                        {'data': {'text/plain': '()'},
                         'execution_count': 7,
                         'metadata': {},
                         'output_type': 'execute_result'}],
            'source': 'println!("Hello Rust")'}],
 'metadata': {},
 'nbformat': 4,
 'nbformat_minor': 5}
